In [10]:
! pip install -q selenium
# https://googlechromelabs.github.io/chrome-for-testing/#stable


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/dc/72/96b5afa16908f9abc7c24b70adfd3a46c9740eb728ddfeab28379e38eaf9/selenium-4.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/3e/14/746bb2b403af4be680ca0ae240d62473c4ec3b836024c2e85f27856d7991/trio-0.23.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
   ---------------------------------------- 0.0/1

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json

driver = webdriver.Chrome(service=Service('helpers/chromedriver-win64/chromedriver.exe'))
driver.get("https://www.adidas.co.in/originals-shoes")
soup = BeautifulSoup(driver.page_source, 'html.parser')
element = soup.find_all('a', {'data-auto-id': 'glass-hockeycard-link'})
hrefs = ["https://www.adidas.co.in"+link.get('href') for link in element]
details = []
for index, link in enumerate(hrefs):
    print(f"trying {index} - {link}")
    link_detail = {"link": link, "brand": "adidas"}
    driver.execute_script("window.open('"+link+"', '_blank')")
    window_name = driver.window_handles[-1]
    driver.switch_to.window(window_name=window_name)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    script_tags = soup.find_all('script')
    data_store_value = None
    for script in script_tags:
        script_content = script.string
        if script_content and 'window.DATA_STORE' in script_content:
            data_store_value = script_content.replace('window.DATA_STORE = JSON.parse(', '').rstrip(');')
            break
    json_value = json.loads(json.loads(data_store_value))
    pid = json_value["nuPdp"]["currentProductId"]
    shortlist =  json_value["productStore"]["products"][pid]["data"]
    rating_details = json_value["nuPdp"]["ratingsReviews"]["ratings"]["value"]["right"]
    for element in rating_details["secondaryRatings"]:
        if element["name"] == "Comfort":
            comfort = element["averageRating"]
        elif element["name"] == "Quality":
            quality = element["averageRating"]
        elif element["name"] == "Width":
            width = element["averageRating"]
        
    link_detail["shoe_name"] =  shortlist["name"]
    link_detail["category"] = shortlist["attribute_list"]["gender"]
    link_detail["style_or_product_code"] = shortlist["model_number"]
    link_detail["reviews"] = rating_details["reviewCount"]
    available_sizes = shortlist["attribute_list"]["sizeTypes"]
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = shortlist["attribute_list"]["color"].split(",")
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    link_detail["stars"] = rating_details["overallRating"]
    link_detail["price"] = shortlist["pricing_information"]["currentPrice"]
    link_detail["quality"], link_detail["comfort"], link_detail["width"] = quality, comfort, width   
    details.append(link_detail)
    driver.close()
    window_name = driver.window_handles[0]
    driver.switch_to.window(window_name=window_name)


print(details)
driver.quit()


[{'link': 'https://www.adidas.co.in/gazelle-shoes/IG1062.html', 'brand': 'adidas', 'shoe_name': 'Gazelle Shoes', 'category': 'U', 'style_or_product_code': 'MDW13', 'reviews': 6, 'number_of_sizes': 0, 'size': '', 'number_of_colors': 1, 'color_1': 'Glory Red / Off White / Cream White', 'color_2': None, 'color_3': None, 'color_4': None, 'color_5': None, 'stars': 4.8, 'price': 10999, 'quality': 5, 'comfort': 4.833333333333333, 'width': 3}, {'link': 'https://www.adidas.co.in/gazelle-bold-shoes/ID6990.html', 'brand': 'adidas', 'shoe_name': 'Gazelle Bold Shoes', 'category': 'W', 'style_or_product_code': 'MBU94', 'reviews': 529, 'number_of_sizes': 0, 'size': '', 'number_of_colors': 1, 'color_1': 'Red / Cloud White / Gold Metallic', 'color_2': None, 'color_3': None, 'color_4': None, 'color_5': None, 'stars': 4.7, 'price': 11999, 'quality': 4.697542533081285, 'comfort': 4.444234404536862, 'width': 3.01890359168242}, {'link': 'https://www.adidas.co.in/samba-og-shoes/B75807.html', 'brand': 'adidas

In [3]:
import pandas as pd
df1_columns = ["shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["reviews", "size", "comfort", "quality", "width", "stars"]
adidas_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in details)
adidas_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in details)
adidas_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in details)

print(adidas_df1)
print("\n")

print(adidas_df2)
print("\n")

print(adidas_df3)
print("\n")

                         shoe_name category  number_of_colors  price
0                    Gazelle Shoes        U                 1  10999
1               Gazelle Bold Shoes        W                 1  11999
2                   Samba OG Shoes        U                 1  10999
3                   Samba OG Shoes        U                 1  11999
4                  Superstar Shoes        U                 1   8999
5                    Gazelle Shoes        U                 1  10999
6                 Stan Smith Shoes        W                 1   9999
7              Superstar XLG Shoes        U                 1  11999
8                       Kick Shoes        M                 1  10999
9                       Kick Shoes        M                 1  10999
10                    Falcon Shoes        W                 1   9999
11                    Falcon Shoes        W                 1   9999
12         Superstar Slip-On shoes        W                 1   9999
13                    Ozelia Shoes